In [1]:
import os
from os.path import dirname, expanduser, join
from subprocess import Popen, PIPE, STDOUT
from IPython.display import HTML

In [2]:
d = dirname(os.getcwd())
print("Build")
p = Popen("bazel build //bindings/pydrake:hack_generate_pybind_documentation_header", shell=True, cwd=d)
assert p.wait() == 0
print(" - Done")

Build
 - Done


In [3]:
mar = open(join(d, "bazel-genfiles/bindings/pydrake/hack_documentation_pybind.h")).read()
lines = mar.split("\n")
values = {"tree": [], "plant": []}
skip = ["MultibodyTree", "MultibodyTree<T>", "MultibodyPlant", "MultibodyPlant<T>"]
mode = None
for line in lines:
    name = line.strip()
    if name == "MultibodyTree":
        mode = "tree"
    elif name == "MultibodyPlant":
        mode = "plant"
    elif name == "":
        mode = None
    elif mode is not None:
        if name not in skip:
            values[mode].append(name)

In [4]:
for mode, items in values.items():
    with open(mode + ".txt", "w") as f:
        f.write("\n".join(items) + "\n")

In [5]:
p = Popen("diff -d -t --side-by-side -W 130 tree.txt plant.txt", shell=True, stdout=PIPE)
assert p.wait() in [0, 1]
text = p.stdout.read()
lines = text.split("\n")

In [6]:
blen = 130 / 2
is_diff = '|'
tree_only = '<'
plant_only = '>'
same = [' ', '\\']

In [7]:
html = "<table>\n"

for line in lines:
    if len(line) == 0:
        continue
    tree = line[:blen - 1].strip()
    sep = line[blen - 1]
    plant = line[blen:].strip()

    if tree == plant + "OrThrow":
        sep = same[0]

    if sep == is_diff:
        # Insert row
        html += "  <tr><td>{}</td><td></td></tr>".format(tree)
        tree = ''
    elif sep == tree_only:
        assert plant == ""
    elif sep == plant_only:
        pass
    elif sep in same:
        fmt = "<font color='green'>{}</font>"
        tree = fmt.format(tree)
        plant = fmt.format(plant)
    else:
        assert False, repr(sep)
    plant = "<i>{}</i>".format(plant)
    html += "  <tr><td>{}</td><td>{}</td></tr>\n".format(tree, plant)

html += "</table>"
HTML(html)

AddBody,
AddForceElement,AddForceElement
AddFrame,AddFrame
AddJoint,AddJoint
AddJointActuator,AddJointActuator
AddMobilizer,
AddModelInstance,AddModelInstance
AddRigidBody,AddRigidBody
CalcAccelerationKinematicsCache,
CalcAcrossNodeGeometricJacobianExpressedInWorld,
CalcAllBodyPosesInWorld,CalcAllBodyPosesInWorld
